In [3]:
import os
import numpy as np

dim = 3
float_precision = 'float32'
root_file = os.getcwd()
mesh_path = 'triangles.msh'
mesh_path = os.path.join(root_file, 'mesh', mesh_path)

def get_members(a):
  return [m for m in dir(a) if not m.startswith("__")]


In [4]:
import importlib

module = importlib.import_module("create_domain")
importlib.reload(module)
Mesh = getattr(module, "Mesh")

mesh = Mesh(mesh_path, dim)

In [21]:
print(get_members(mesh.mesh))
print(get_members(mesh.mesh.cells[4]))
print(mesh.cells_dict)

['cell_data', 'cell_data_dict', 'cell_data_to_sets', 'cell_sets', 'cell_sets_dict', 'cell_sets_to_data', 'cells', 'cells_dict', 'copy', 'field_data', 'get_cell_data', 'get_cells_type', 'gmsh_periodic', 'info', 'point_data', 'point_data_to_sets', 'point_sets', 'point_sets_to_data', 'points', 'read', 'write']
['data', 'dim', 'tags', 'type']


AttributeError: 'Mesh' object has no attribute 'cells_dict'

In [ ]:
import os
import numpy as np

dim = 3
float_precision = 'float32'
root_file = os.getcwd()
mesh_path = 'tetrahedron.msh'
mesh_path = os.path.join(root_file, 'mesh', mesh_path)

def get_members(a):
  return [m for m in dir(a) if not m.startswith("__")]

import importlib

module_1 = importlib.import_module("create_domain_utils")
importlib.reload(module_1)
module = importlib.import_module("create_domain")
importlib.reload(module)
Mesh = getattr(module, "Mesh")
Domain = getattr(module, "Domain")
LocalDomain = getattr(module, "LocalDomain")

mesh = Mesh(mesh_path, dim)
domain = Domain(mesh, float_precision)
print(domain.cells.shape[0])


local_domains_data = domain.c_create_sub_domains(4)
# new_nb_part = len(local_domains_data)

# PC
# 2m 2s -> 30000 (nid)
# 37s -> 4 (nid)
# c 10s -> 4 (fid)
# c 40s -> 30000 (fid)
# c 1m 42s -> 30000 (nid)
# ---- laptop
# 1m 44s -> 4 (nid)
# print(local_domains_data)


In [ ]:
print(local_domains_data[0].nodes.shape[0])

In [ ]:
from manapy.partitions import MeshPartition
from manapy.base.base import Struct

running_conf = Struct(backend="numba", signature=True, cache=True, float_precision=float_precision)
a = MeshPartition(mesh_path, dim=dim, conf=running_conf, periodic=[0,0,0])

# 6m 49 -> 30000
# 2m -> 3000
# 28s -> 4
# ----
# 1m 26s -> 4


In [ ]:
for i in range(domain.cells.shape[0]):
  c = domain.cells[i]
  if 110 in c and 363 in c:
    print(i, c)

In [ ]:
domain.node_phyfaceid[111]

In [ ]:
c = 5699
print(domain.cells[c])
r = domain.cell_faceid[5699]
print(r)
print(domain.faces[r[0:-1]])

In [ ]:
a = domain.node_phyfaceid
print(a[0:100])

In [ ]:
a = domain.face_name
print(len(a[a == 2]))

In [ ]:
for item in domain.phy_faces:
  a = item[0:item[-1]-1]
  a = np.sort(a)
  #print(a)
  if np.all(a == np.array([100, 101])):
    print(item)

In [ ]:
for p in range(4):
  a = local_domains[p].map_nodes.get(110)
  print(a)

In [ ]:
c = 0
for p in range(4):
  a = local_domains[p].map_phy_faces.values()
  a = np.array(list(a))
  a.sort()
  #print(a)
  print(len(a))
  c += len(a)
print(c)



In [ ]:
a = {}
a[0] = len(a)
print(a)

In [ ]:
import importlib

module = importlib.import_module("create_domain")
importlib.reload(module)
Domain = getattr(module, "Domain")

domain = Domain(mesh_path, dim, float_precision)
print(domain.cells.shape)

In [ ]:
print(domain.cell_cellfid)

In [ ]:
# 511104 -> 1.3s
# 5821794 -> 17.9s
members = [m for m in dir(domain.meshio_mesh) if not m.startswith("__")]
print(members)

print(domain.meshio_mesh.cell_data)

In [ ]:
print(domain.meshio_mesh.cell_data['tetra']['gmsh:physical'])

In [ ]:
from manapy.partitions import MeshPartition
from manapy.base.base import Struct
from manapy.ddm import Domain

def create_partitions(mesh_file_path, float_precision, dim):
  running_conf = Struct(backend="numba", signature=True, cache=True, float_precision=float_precision)
  MeshPartition(mesh_file_path, dim=dim, conf=running_conf, periodic=[0,0,0])
  domain = Domain(dim=dim, conf=running_conf)
  return domain

domain = create_partitions(mesh_path, float_precision, dim)

In [ ]:
domain.faces.nodeid.shape[0]

In [ ]:
domain.meshio_mesh.cell_data['tetra']['gmsh:physical'][0::10]